<a href="https://colab.research.google.com/github/Rakabi007/Smart-Scholarship-Allocation-Process-with-Optimization-and-Eligibility-Matching/blob/main/Smart_Scholarship_Allocation_Process_with_Optimization_and_Eligibility_Matchingz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Why would we be interested in doing smart scholarship allocation with optimization and eligibility matching?

# Maximize Impact With Limited Funds
# Guarantee Fairness, Transparency & Equity
# Handle Massive Applicant Volumes
# Enforce Complex Scholarship Rules Automatically
# Prevent Over-Awarding or Double-Awarding
# Align Scholarships With Institutional Goals
# Quantify and Measure Fairness
# Build Trust With Donors
# Increase Student Satisfaction
# Faster Awarding > Higher College Enrollment
# In short: Smart Scholarship Optimization = Maximum Fairness, Maximum Impact, Minimum Headache


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)        # Display all rows
pd.set_option('display.max_columns', None)     # Display all columns


rng = np.random.default_rng(42)


# Students Dataset

N_STUDENTS = 200

majors = ["CS", "Business", "Engineering", "Biology", "Economics"]
income_brackets = ["low", "mid", "high"]

df_students = pd.DataFrame({
    "student_id": [f"S{i:03d}" for i in range(N_STUDENTS)],
    "gpa": np.round(rng.normal(3.2, 0.4, N_STUDENTS).clip(2.0, 4.0), 2),
    "test_score": rng.integers(900, 1601, N_STUDENTS),   # 900–1600 like SAT
    "income_bracket": rng.choice(income_brackets, size=N_STUDENTS, p=[0.3, 0.5, 0.2]),
    "major": rng.choice(majors, size=N_STUDENTS),
})

# Need: low income → higher need score
need_map = {"low": 1.0, "mid": 0.5, "high": 0.1}
df_students["need_score"] = df_students["income_bracket"].map(need_map)

# Merit: scaled combination of GPA + test_score
gpa_norm = (df_students["gpa"] - df_students["gpa"].min()) / (df_students["gpa"].max() - df_students["gpa"].min())
test_norm = (df_students["test_score"] - df_students["test_score"].min()) / (df_students["test_score"].max() - df_students["test_score"].min())

df_students["merit_score"] = 0.6 * gpa_norm + 0.4 * test_norm

df_students.head()

,student_id,gpa,test_score,income_bracket,major,need_score,merit_score
0,S000,3.32,1554,high,Biology,0.1,0.727908
1,S001,2.78,909,mid,Engineering,0.5,0.159250
2,S002,3.50,1022,mid,Business,0.5,0.486291
3,S003,3.58,1060,mid,Biology,0.5,0.537316
4,S004,2.42,1095,low,Business,1.0,0.135700


In [2]:
# Define Scholarships Dataset

df_sch = pd.DataFrame([
    {"sch_id": "SCH_NEED_1", "name": "Access Grant I", "amount": 5000, "max_awards": 15,
     "min_gpa": 2.8, "target_major": "ANY", "need_weight": 0.8, "merit_weight": 0.2},
    {"sch_id": "SCH_NEED_2", "name": "Access Grant II", "amount": 3000, "max_awards": 25,
     "min_gpa": 2.5, "target_major": "ANY", "need_weight": 0.7, "merit_weight": 0.3},
    {"sch_id": "SCH_MERIT_1", "name": "Excellence Award I", "amount": 8000, "max_awards": 10,
     "min_gpa": 3.7, "target_major": "ANY", "need_weight": 0.3, "merit_weight": 0.7},
    {"sch_id": "SCH_CS", "name": "CS Innovators", "amount": 6000, "max_awards": 8,
     "min_gpa": 3.3, "target_major": "CS", "need_weight": 0.4, "merit_weight": 0.6},
    {"sch_id": "SCH_ENG", "name": "Engineering Leaders", "amount": 6000, "max_awards": 8,
     "min_gpa": 3.3, "target_major": "Engineering", "need_weight": 0.4, "merit_weight": 0.6},
])

display(df_sch)




,sch_id,name,amount,max_awards,min_gpa,target_major,need_weight,merit_weight
0,SCH_NEED_1,Access Grant I,5000,15,2.8,ANY,0.8,0.2
1,SCH_NEED_2,Access Grant II,3000,25,2.5,ANY,0.7,0.3
2,SCH_MERIT_1,Excellence Award I,8000,10,3.7,ANY,0.3,0.7
3,SCH_CS,CS Innovators,6000,8,3.3,CS,0.4,0.6
4,SCH_ENG,Engineering Leaders,6000,8,3.3,Engineering,0.4,0.6


In [3]:
# Build eligibility & “value” for each student–scholarship pair
# Cartesian join of students × scholarships

df_pairs = df_students.assign(key=1).merge(df_sch.assign(key=1), on="key", suffixes=("_stu", "_sch")).drop(columns=["key"])

# Eligibility rules
def is_eligible(row):
    if row["gpa"] < row["min_gpa"]:
        return 0
    if row["target_major"] != "ANY" and row["major"] != row["target_major"]:
        return 0
    return 1

df_pairs["eligible"] = df_pairs.apply(is_eligible, axis=1)
df_pairs["eligible"].value_counts()

display(df_pairs.head())


# value = amount × (need_weight * need_score + merit_weight * merit_score)

df_pairs["match_value"] = (
    df_pairs["amount"] *
    (df_pairs["need_weight"] * df_pairs["need_score"] +
     df_pairs["merit_weight"] * df_pairs["merit_score"])
)

df_pairs.head()

,student_id,gpa,test_score,income_bracket,major,need_score,merit_score,sch_id,name,amount,max_awards,min_gpa,target_major,need_weight,merit_weight,eligible
0,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_NEED_1,Access Grant I,5000,15,2.8,ANY,0.8,0.2,1
1,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_NEED_2,Access Grant II,3000,25,2.5,ANY,0.7,0.3,1
2,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_MERIT_1,Excellence Award I,8000,10,3.7,ANY,0.3,0.7,0
3,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_CS,CS Innovators,6000,8,3.3,CS,0.4,0.6,0
4,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_ENG,Engineering Leaders,6000,8,3.3,Engineering,0.4,0.6,0


,student_id,gpa,test_score,income_bracket,major,need_score,merit_score,sch_id,name,amount,max_awards,min_gpa,target_major,need_weight,merit_weight,eligible,match_value
0,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_NEED_1,Access Grant I,5000,15,2.8,ANY,0.8,0.2,1,1127.907648
1,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_NEED_2,Access Grant II,3000,25,2.5,ANY,0.7,0.3,1,865.116883
2,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_MERIT_1,Excellence Award I,8000,10,3.7,ANY,0.3,0.7,0,4316.282828
3,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_CS,CS Innovators,6000,8,3.3,CS,0.4,0.6,0,2860.467532
4,S000,3.32,1554,high,Biology,0.1,0.727908,SCH_ENG,Engineering Leaders,6000,8,3.3,Engineering,0.4,0.6,0,2860.467532


In [4]:
# Optimization model: maximize impact subject to constraints

import numpy as np
import pandas as pd
import pulp as pl   # ← add this

# ... your df_students, df_sch, df_pairs code ...

df_elig = df_pairs[df_pairs["eligible"] == 1].copy()
print("Eligible pairs:", len(df_elig))

prob = pl.LpProblem("Scholarship_Allocation", pl.LpMaximize)

x = {
    idx: pl.LpVariable(f"x_{idx}", lowBound=0, upBound=1, cat="Binary")
    for idx in df_elig.index
}

prob += pl.lpSum(df_elig.loc[idx, "match_value"] * x[idx] for idx in df_elig.index)

for sch_id, group in df_elig.groupby("sch_id"):
    max_aw = group["max_awards"].iloc[0]
    prob += pl.lpSum(x[idx] for idx in group.index) <= max_aw, f"cap_{sch_id}"

MAX_SCH_PER_STUDENT = 1

for stu_id, group in df_elig.groupby("student_id"):
    prob += pl.lpSum(x[idx] for idx in group.index) <= MAX_SCH_PER_STUDENT, f"stu_limit_{stu_id}"


prob.solve(pl.PULP_CBC_CMD(msg=False))
print("Status:", pl.LpStatus[prob.status])




Eligible pairs: 418
Status: Optimal


In [5]:
# Optional fairness: at least N% of awards to low-income students

TOTAL_AWARDS = df_sch["max_awards"].sum()

low_income_idx = df_elig[df_elig["income_bracket"] == "low"].index

prob += pl.lpSum(x[idx] for idx in low_income_idx) >= 0.4 * TOTAL_AWARDS, "low_income_min_share"


# Solve the model

prob.solve(pl.PULP_CBC_CMD(msg=False))

print("Status:", pl.LpStatus[prob.status])

chosen_idxs = [idx for idx, var in x.items() if var.value() == 1]
df_alloc = df_elig.loc[chosen_idxs].copy()

print("Total awards allocated:", len(df_alloc))
df_alloc.head()

Status: Optimal
Total awards allocated: 66


,student_id,gpa,test_score,income_bracket,major,need_score,merit_score,sch_id,name,amount,max_awards,min_gpa,target_major,need_weight,merit_weight,eligible,match_value
40,S008,3.19,1579,low,Business,1.0,0.695065,SCH_NEED_1,Access Grant I,5000,15,2.8,ANY,0.8,0.2,1,4695.064935
58,S011,3.51,1254,mid,CS,0.5,0.623838,SCH_CS,CS Innovators,6000,8,3.3,CS,0.4,0.6,1,3445.818182
68,S013,3.65,1386,mid,CS,0.5,0.750938,SCH_CS,CS Innovators,6000,8,3.3,CS,0.4,0.6,1,3903.376623
70,S014,3.39,1437,low,Economics,1.0,0.685830,SCH_NEED_1,Access Grant I,5000,15,2.8,ANY,0.8,0.2,1,4685.829726
93,S018,3.55,1505,mid,CS,0.5,0.783261,SCH_CS,CS Innovators,6000,8,3.3,CS,0.4,0.6,1,4019.740260


In [6]:
# Analyze results

# 1) Check per-scholarship awards
df_alloc.groupby("sch_id")["student_id"].nunique()


# Compare to max_awards:

df_sch[["sch_id", "max_awards"]]


# Check per-student count

df_alloc.groupby("student_id")["sch_id"].nunique().describe()


# Check fairness: low-income share

low_share = (df_alloc["income_bracket"] == "low").mean()
print("Low-income share of winners:", round(low_share, 3))

Low-income share of winners: 0.758


In [7]:
# Look at top recipients by total award

df_alloc["total_award"] = df_alloc["amount"]

winners = (
    df_alloc.groupby(["student_id", "income_bracket", "major"])
            .agg(total_award=("total_award", "sum"),
                 avg_gpa=("gpa", "mean"),
                 avg_merit=("merit_score", "mean"))
            .reset_index()
            .sort_values("total_award", ascending=False)
)

print(winners.shape)
winners.head()

(66, 6)


,student_id,income_bracket,major,total_award,avg_gpa,avg_merit
63,S189,mid,Economics,8000,3.86,0.861934
17,S057,low,Business,8000,3.80,0.915729
53,S162,mid,Biology,8000,4.00,0.981530
35,S118,low,Economics,8000,3.84,0.912958
64,S190,mid,Economics,8000,3.89,0.811659


In [8]:
# How scholarships are used (5 scholarships)

sch_summary = (
    df_alloc.groupby(["sch_id", "name"])
            .agg(
                awards=("student_id", "nunique"),
                avg_gpa=("gpa", "mean"),
                low_income_share=("income_bracket", lambda s: (s == "high").mean()),
                avg_need=("need_score", "mean"),
                avg_merit=("merit_score", "mean"),
                total_amount=("amount", "sum")
            )
            .reset_index()
)

print(sch_summary.shape)
sch_summary




(5, 8)


,sch_id,name,awards,avg_gpa,low_income_share,avg_need,avg_merit,total_amount
0,SCH_CS,CS Innovators,8,3.54500,0.0,0.625,0.684473,48000
1,SCH_ENG,Engineering Leaders,8,3.60375,0.0,0.750,0.609300,48000
2,SCH_MERIT_1,Excellence Award I,10,3.87000,0.0,0.700,0.868283,80000
3,SCH_NEED_1,Access Grant I,15,3.28000,0.0,1.000,0.681424,75000
4,SCH_NEED_2,Access Grant II,25,3.10960,0.0,1.000,0.464109,75000
